In [1]:
# GPU Testing and print related info
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
else:
    print(device)

NVIDIA GeForce RTX 4080 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
# Upload dataset csv file to python:
import numpy as np
import pandas as pd
file_path = "./Database/New/stocks/A.csv"
stock = pd.read_csv(file_path)
stock['Index'] = stock.reset_index().index
for column in stock:
    print(column)
print(stock.shape)
print(stock)


Date
Open
High
Low
Close
Adj Close
Volume
Index
(6182, 8)
            Date        Open        High         Low       Close   Adj Close  \
0     11/18/1999   32.546494   35.765381   28.612303   31.473534   26.652395   
1     11/19/1999   30.713518   30.758226   28.478184   28.880545   24.456598   
2     11/22/1999   29.551144   31.473534   28.657009   31.473534   26.652395   
3     11/23/1999   30.400572   31.205294   28.612303   28.612303   24.229448   
4     11/24/1999   28.701717   29.998213   28.612303   29.372318   24.873045   
...          ...         ...         ...         ...         ...         ...   
6177   6/10/2024  132.869995  133.850006  132.250000  133.029999  133.029999   
6178   6/11/2024  132.639999  132.990005  131.490005  132.110001  132.110001   
6179   6/12/2024  133.630005  133.639999  132.330002  133.160004  133.160004   
6180   6/13/2024  132.589996  133.000000  130.039993  130.570007  130.570007   
6181   6/14/2024  130.000000  130.740005  129.240005  129.8500

In [13]:
# Separate the original stock dataset into a number of training blocks
n = len(stock)
rows_per_dataset = n // 78
remainder = n % 78
stock_set = []

start_idx = 0
for i in range(78):
    if i < remainder:
        end_idx = start_idx + rows_per_dataset + 1
    else:
        end_idx = start_idx + rows_per_dataset
    stock_bloc = stock.iloc[start_idx : end_idx]
    start_idx = end_idx
    stock_set.append(stock_bloc)

In [16]:
# Select specific columns from a single stock:
C = stock_set[1]["High"]
t = stock_set[1]["Index"]
print(stock)


Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, Index]
Index: []


In [17]:
# Algorithm 1:
import math as m
p = [None]*79
u = [None]*79
d = [None]*79
C_pred = [None]*79
for k in t:
    C_pred[k+1] = C[k]*u[k]*p[k] + C[k]*d[k]*(1-p[k])
    # Absolute value means L1-Distance, but also could use L2-Distance.
    diff = m.abs(C_pred[k+1] - C[k+1])


In [18]:
# Algorithm 2:
p = [None]*79
n = [None]*79
C_sub = [None]*79
C_pred = [None]*79
for i in t:
    for j in range(n[i]):
        C_sub[i+j+1] = (C_sub[i+j] + 1*(1/(m.sqrt(n))))*p[i]
        + (C_sub[i+j] + (-1)*(1/(m.sqrt(n))))*(1-p[i])
    C_pred[i+1] = C_sub[i+j+1]
    # L1-Distance
    diff = m.abs(C_pred[i+1] - C[i+1])

In [ ]:
# RNN Training:
import tensorflow as tf

# ... 